# Page Visits Funnel

Cool T-Shirts Inc. has asked you to analyze data on visits to their website. Your job is to build a funnel, which is a description of how many people continue to the next step of a multi-step process.

In this case, our funnel is going to describe the following process:

1. A user visits CoolTShirts.com
2. A user adds a t-shirt to their cart
3. A user clicks “checkout”
4. A user actually purchases a t-shirt

In [40]:
import pandas as pd

Import all the files

In [41]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [42]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [43]:
visits_cart_merged = pd.merge(visits, cart, how="left")

Step 3: How long is `visits_cart`?

In [44]:
print(len(visits_cart_merged))

2000


Step 4: How many timestamps are null for `cart_time`?

In [45]:
no_add_to_cart = visits_cart_merged[visits_cart_merged.cart_time.isnull()]
print(len(no_add_to_cart))

1652


Step 5: What percentage only visited?

In [46]:
pct_no_add_to_cart = len(no_add_to_cart)/float(len(visits_cart_merged))
print(pct_no_add_to_cart)

0.826


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [47]:
cart_checkout_merged = pd.merge(cart, checkout, how="left")
display(cart_checkout_merged.head().style)
# added item, didn't checkout
add_item_no_checkout = cart_checkout_merged[cart_checkout_merged.checkout_time.isnull()]
# percentage of users who added to cart, didn't checkout
pct_added_no_checkout = float(len(add_item_no_checkout))/len(cart)
print(pct_added_no_checkout)

,user_id,cart_time,checkout_time
0,2be90e7c-9cca-44e0-bcc5-124b945ff168,2017-11-07 20:45:00,2017-11-07 21:14:00
1,4397f73f-1da3-4ab3-91af-762792e25973,2017-05-27 01:35:00,NaT
2,a9db3d4b-0a0a-4398-a55a-ebb2c7adf663,2017-03-04 10:38:00,2017-03-04 11:04:00
3,b594862a-36c5-47d5-b818-6e9512b939b3,2017-09-27 08:22:00,2017-09-27 08:26:00
4,a68a16e2-94f0-4ce8-8ce3-784af0bbb974,2017-07-26 15:48:00,NaT


0.3505747126436782


Step 7: Merge it all together

In [48]:
all_data = visits.merge(cart, how="left").merge(checkout, how="left").merge(purchase, how="left")
display(all_data.head().style)

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT


Step 8: % of users who got to checkout but did not purchase

In [49]:
reached_checkout = all_data[~all_data.purchase_time.isnull()]
checkout_not_purchase = all_data[(all_data.purchase_time.isnull()) & (~all_data.checkout_time.isnull())]
pct_checkout_no_purchase = len(checkout_not_purchase) / float(len(reached_checkout))

Step 9: check each part of the funnel, let's print all 3 of them again

In [50]:
print("percentage of users who visited Cool T-Shirts Inc. but did not place a t-shirt in their cart {}".format(pct_no_add_to_cart*100))
print("percentage of users put items in their cart, but did not proceed to checkout {}".format(pct_added_no_checkout*100))
print("percentage of users proceeded to checkout, but did not purchase a t-shirt {}".format(pct_checkout_no_purchase*100))

percentage of users who visited Cool T-Shirts Inc. but did not place a t-shirt in their cart 82.6
percentage of users put items in their cart, but did not proceed to checkout 35.05747126436782
percentage of users proceeded to checkout, but did not purchase a t-shirt 32.53968253968254


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [51]:
all_data['time_to_purchase'] = all_data['purchase_time'] - all_data['visit_time']

Step 11: examine the results

In [52]:
print(all_data['time_to_purchase'])

0                  NaT
1      0 days 00:44:00
2                  NaT
3                  NaT
4                  NaT
             ...      
2103               NaT
2104               NaT
2105               NaT
2106               NaT
2107               NaT
Name: time_to_purchase, Length: 2108, dtype: timedelta64[ns]


Step 12: average time to purchase

In [53]:
average_purchase_time = all_data.time_to_purchase.mean()
print("average purchase time {}".format(average_purchase_time))

average purchase time 0 days 00:43:12.380952380
